# Circular Molecules

> ### In this tutorial we will cover:
> - how we can build circular structures in biobuild

Sometimes, the molecules we want to build are circular. This poses a challenge to biobuild which is tailored to linear molecules. In fact, biobuild's rotational optimization algorithms for improving conformations are useless for circular structures. Hence, in pure biobuild we cannot build circular molecules at this point!

For small rings it is usually possible to find reference components that already contain these rings, allowing users to circumvent the problem altogether by smart choices of starting compounds. 

However, didn't the tutorial title suggest that we _could_ build circular structures? Well, yes, we can - it's just not something that biobuild _inherently_ can do. But this is where biobuild's excellent integratability with other libraries comes in handy. 

The cheminformatics library `RDKit` uses a molecular force-field optimization that is perfectly able to handle ring-like structures. So, in order to build circular structures in biobuild, we (1) create the structure we want linearly, (2) "circularize" it by adding the necessary bonds, and (3) export it to RDKit to perform the conformational optimization. If after cricularization our building process is not yet complete, we can always (4) import the molecule back into biobuild to continue our work.

### Example

Let's make a super simple circular poly-histidine.

In [ ]:
import plotly
plotly.offline.init_notebook_mode()

In [1]:
import buildamol as bam

# load the amino acids
bam.load_amino_acids()

# make a histidine
his = bam.molecule("HIS")

# and make a polymer of 50 histidines
his.repeat(50, "LINK")

his.show(residue_graph=True)

Now that we have our linear poly-histidine, let's add a "circular" bond by connecting the first residue to the last residue. We cannot use `bam.connect` or the `attach` method for this, however, because these methods would transpose the atom coordinates in our molecule, completely twisting the structure. But it's no trouble to manually add one bond and remove two atoms. So let's do it.

In [2]:
# remove the HA atom from the first histidine
# and the HXT atom from the last histidine
his.remove_atoms(
    his.get_atom("HA", residue=1),
    his.get_atom("HXT", residue=-1),
)

[Atom(HA, 13), Atom(HXT, 903)]

In [3]:
# now connect the CA from the first histidine
# to the OXT from the last histidine
his.add_bond(
    his.get_atom("CA", residue=1),
    his.get_atom("OXT", residue=-1),
)


If we look at the residue graph again, we will now see a new very looong bond spanning the entire length of the polymer.

In [4]:
his.show(residue_graph=True)

This is where our biobuild use comes to a temporary end, and we make use of RDKit to handle the circularization part more properly.

In [7]:
# import AllChem from rdkit
from rdkit.Chem import AllChem

# export the molecule to RDKit
rdkit_mol = his.to_rdkit()

# use RDKit to improve the conformation
AllChem.EmbedMolecule(rdkit_mol)
AllChem.UFFOptimizeMolecule(rdkit_mol)

# import the molecule back into biobuild
optimized_his = bam.molecule(rdkit_mol)

# show the molecule
optimized_his.show(residue_graph=True)

And there it is, a nice circle of histidines. We can also use nglview to get a more detailed view.

So, that's it for this tutorial. Building circular structures is not so difficult after all, but it does involve making use of biobuild's integratability features. Thank you for checking out this tutorial and good luck with your research!